In [59]:
import pandas as pd
import numpy as np
from openai import OpenAI

In [61]:
df = pd.read_csv('../data/data.csv')
df = df[['text', 'transparency_sum', 'dignity_sum', 'sustainability_sum', 'trust_sum','freedom_autonomy_sum', 'beneficence_sum', 'privacy_sum', 'responsibility_sum', 'non_maleficence_sum', 'justice_equity_sum']]
df = df.rename(columns={'transparency_sum': 'transparency', 'dignity_sum': 'dignity', 'sustainability_sum': 'sustainability', 'trust_sum': 'trust', 'freedom_autonomy_sum': 'freedom and autonomy', 'beneficence_sum': 'beneficence', 'privacy_sum': 'privacy', 'responsibility_sum': 'responsibility', 'non_maleficence_sum': 'non-maleficence', 'justice_equity_sum': 'justice and equity'})
df.columns

Index(['Unnamed: 0', 'text', 'major_principle', 'is_major_principle_here',
       'ethical_us', 'correct'],
      dtype='object')

In [3]:
for i, row in df.iterrows():
    row = row[1:]
    if row.sum() == 0.0:
        df.at[i,'major_principle'] = None
        df.at[i,'is_major_principle_here'] = False
    else:
        df.at[i,'major_principle'] = row.idxmax()
        df.at[i,'is_major_principle_here'] = True
df.head()

,text,transparency,dignity,sustainability,trust,freedom and autonomy,beneficence,privacy,responsibility,non-maleficence,justice and equity,major_principle,is_major_principle_here
0,An intelligent system connected to the video s...,2.0,0.0,0.0,2.0,0.0,0.0,3.0,2.0,0.0,1.0,privacy,True
1,Terminal area and booking documents and bookin...,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,transparency,True
2,Before a person with a vehicle enters the port...,5.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,0.0,1.0,transparency,True
3,A clear advertisement is placed on the booking...,5.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,transparency,True
4,Data information policy is communicated at the...,5.0,1.0,0.0,3.0,2.0,1.0,4.0,4.0,1.0,1.0,transparency,True


In [4]:
df = df[['text', 'major_principle', 'is_major_principle_here']]
df['ethical_us'] = 'empty'
df

,text,major_principle,is_major_principle_here,ethical_us
0,An intelligent system connected to the video s...,privacy,True,empty
1,Terminal area and booking documents and bookin...,transparency,True,empty
2,Before a person with a vehicle enters the port...,transparency,True,empty
3,A clear advertisement is placed on the booking...,transparency,True,empty
4,Data information policy is communicated at the...,transparency,True,empty
...,...,...,...,...
1088,The website must provide highest degree of sec...,None,False,empty
1089,The product shall be intuitive and self-explan...,None,False,empty
1090,The system shall be intuitive and self explana...,None,False,empty
1091,The product shall use symbols and words that a...,None,False,empty


In [5]:
client = OpenAI()

for i, row in df.iterrows():
  if row['major_principle'] is not None:
    messages = [
        {"role": "system", "content": "You are an ethical requirements engineer translating requirements to ethical user stories based on one of the 10 ethical principles:\n"},
        {"role": "system", "content": "Transparency: Transparency can typically be understood in two ways: the transparency of the AI technology itself and the transparency of the AI organisations developing and using it. Throughout our analysis, transparency was regularly discussed directly, or in relation to processes required to ensure it, such as explainability, understandability and communication.\n"},
        {"role": "system", "content": "Non-maleficence: The principle of non-maleficence gained attention, and in its most basic form, it means to do no harm or avoid doing harm to others\n"},
        {"role": "system", "content": "Responsibility: The principle of responsibility is base on responsibility of the developers and stakeholders over the AI, accountability, liability and acting with integrity\n"},
        {"role": "system", "content": "Privacy: Related to privacy of personal data, because of the large abundance of data that is required for AI to work, it is important that individuals privacy is not jeopardised as a result\n"},
        {"role": "system", "content": "Beneficence: Beneficence essentially means to do good, to carry out an activity with the intention of benefitting someone or society as a whole\n"},
        {"role": "system", "content": "Freedom and autonomy: Democratic societies place value in freedom and autonomy, and it is important that AI use does not encumber or harm these for us\n"},
        {"role": "system", "content": "Sustainability: All fields and disciplines are affected and need to incorporate sustainability agendas\n"},
        {"role": "system", "content": "Dignity: Human dignity is the recognition that individuals have inherent worth and that their rights should be respected\n"},
        {"role": "system", "content": "Justice and fairness: The issue of discrimination and unfair results resulting from algorithms has become a significant concern. It is imperative that systems are designed to ensure that they are free from any form of unfairness and inequality.\n"},
        {"role": "system", "content": "Trust: Trust is built by keeping promises, making sure systems work properly and protecting data responsibly. Organisations must prove their trustworthiness by ensuring that their technologies are secure and effective.\n"},
        {"role": "system", "content": "and the TEMPLATE:\n As a <persona> i want to <do something> <so that> \n\n"}, 
        {"role": "user", "content": f"Now following this template and the ethical principles definition, transform the requirement {row['text']} into brief a description of an ethical user story based on the principle {row['major_principle']}, substituting <persona> for the one that requires this functionality, <do something> for what the <persona> wants to do and <so that> to the end goal of the functionality"}
  ]
    response = client.chat.completions.create(
      model="gpt-4o",
      messages=messages
    )
    df.at[i,'ethical_us'] = response.choices[0].message.content

  else:
    messages = [
        {"role": "system", "content": "You are an ethical requirements engineer translating requirements to ethical user stories based on one of the 10 ethical principles:\n"},
        {"role": "system", "content": "Transparency: Transparency can typically be understood in two ways: the transparency of the AI technology itself and the transparency of the AI organisations developing and using it. Throughout our analysis, transparency was regularly discussed directly, or in relation to processes required to ensure it, such as explainability, understandability and communication.\n"},
        {"role": "system", "content": "Non-maleficence: The principle of non-maleficence gained attention, and in its most basic form, it means to do no harm or avoid doing harm to others\n"},
        {"role": "system", "content": "Responsibility: The principle of responsibility is base on responsibility of the developers and stakeholders over the AI, accountability, liability and acting with integrity\n"},
        {"role": "system", "content": "Privacy: Related to privacy of personal data, because of the large abundance of data that is required for AI to work, it is important that individuals privacy is not jeopardised as a result\n"},
        {"role": "system", "content": "Beneficence: Beneficence essentially means to do good, to carry out an activity with the intention of benefitting someone or society as a whole\n"},
        {"role": "system", "content": "Freedom and autonomy: Democratic societies place value in freedom and autonomy, and it is important that AI use does not encumber or harm these for us\n"},
        {"role": "system", "content": "Sustainability: All fields and disciplines are affected and need to incorporate sustainability agendas\n"},
        {"role": "system", "content": "Dignity: Human dignity is the recognition that individuals have inherent worth and that their rights should be respected\n"},
        {"role": "system", "content": "Justice and fairness: The issue of discrimination and unfair results resulting from algorithms has become a significant concern. It is imperative that systems are designed to ensure that they are free from any form of unfairness and inequality.\n"},
        {"role": "system", "content": "Trust: Trust is built by keeping promises, making sure systems work properly and protecting data responsibly. Organisations must prove their trustworthiness by ensuring that their technologies are secure and effective.\n"},
        {"role": "system", "content": "and the TEMPLATE:\n As a <persona> i want to <do something> <so that> \n\n"}, 
        {"role": "user", "content": f"Now following this template and the ethical principles definition choose one of the ethical principles and transform the requirement below into a brief description of an ethical user story based on the choosen ethical principle, substituting <persona> for the one that requires this functionality, <do something> for what the <persona> wants to do and <so that> to the end goal of the functionality"}
    ]

    response = client.chat.completions.create(
      model="gpt-4o",
      messages=messages
    )
    df.at[i,'ethical_us'] = response.choices[0].message.content


In [62]:
df['correct'] = df['ethical_us'].str.contains('As a')
df['correct'].value_counts()

correct
True    927
Name: count, dtype: int64

In [63]:
df['ethical_us'] = df['ethical_us'].str.extract(r'(As a.*)')

In [64]:
df['correct'] = df['ethical_us'].str.contains('As a')
df = df[df['correct']]
df = df.drop(columns=['Unnamed: 0'])
df['correct'].value_counts()

correct
True    927
Name: count, dtype: int64

In [66]:
df.to_csv('../data/data_final.csv')